In [0]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [0]:
from fastai import *
from fastai.text import *

In [0]:
!unzip Data.zip

In [0]:
path = '/content/Data'
data_lm = (TextList.from_folder(path)
          .split_by_rand_pct(valid_pct=0)
          .label_for_lm()
          .databunch())

In [0]:
data_lm.show_batch()

idx,text
0,"impurezas ; \n xxup ii xxunk de retenção ou de referência - amostra de intermediário ou de excipiente farmacêutico , xxunk pelo fabricante , devidamente identificada para avaliação futura da qualidade do lote ; \n xxup iii -área - espaço físico delimitado , onde são realizadas operações sob condições ambientais específicas , quando aplicável ; \n xxup iv xxunk - demonstração que um instrumento ou sistema de"
1,"os equipamentos estejam calibrados e \n qualificados e a manutenção seja realizada . \n xxmaj seção xxup iv \n xxmaj revisão da qualidade \n xxmaj art . 22 ° xxmaj revisões regulares da qualidade dos insumos farmacêuticos \n ativos devem ser conduzidas pelo menos anualmente , com \n o objetivo de verificar a consistência do processo . \n xxmaj art . 23 °"
2,", de 5 de janeiro de 2018 ) \n \n xxmaj art . 5º xxmaj no ato do desembaraço aduaneiro pela autoridade sanitária local , nos portos e aeroportos relacionados no xxmaj anexo xxup ii da presente xxmaj resolução , todos os lotes serão submetidos à análise de controle de qualidade quanto a atividade específica , ensaios químicos , sorológicos e documental . xxmaj para tanto a xxmaj gerência"
3,"para cada hemocomponente , de forma ordenada e racional . \n § 1 ° xxmaj os reagentes devem ser estocados em refrigeradores apropriados , os quais devem conservar apenas reagentes e amostras laboratoriais , salvo para serviços de hemoterapia com pequeno número de transfusões , pequeno o xxunk de modo que o armazenamento conjunto com hemocomponentes não comprometa a conservação dos mesmos . \n § 2 ° xxmaj"
4,requisitos xxup para xxup aquisição e xxup qualificação xxup das xxup embalagens xxup para xxup acondicionamento xxup de xxup tecidos xxup humanos \n xxmaj seção i \n xxmaj disposições gerais \n xxmaj art . 5º xxmaj as embalagens primárias utilizadas no acondicionamento de tecidos humanos para fins terapêuticos não são passíveis de registro ou cadastro pela xxmaj anvisa . \n xxmaj parágrafo único . xxmaj excetuam


In [0]:
learn_lm = language_model_learner(data_lm, AWD_LSTM, drop_mult = 0.5, pretrained=False, callback_fns=ShowGraph)

In [0]:
learn_lm.fit_one_cycle(10,2e-2,moms=(0.8,0.7))

In [0]:
import pickle

with open('res.txt','rb') as fp: resolucoes = pickle.load(fp)

In [0]:
'''
mat = np.zeros(1152)
for resolucao in resolucoes:
  xb,yb = learn_lm.data.one_item(resolucao)
  sentence = torch.cuda.LongTensor(xb.tolist()[0]).unsqueeze(0)
  raw_outputs, outputs = learn_lm.model[0].forward(sentence)
  sentence_rep = outputs[0][-1][-1].cpu().detach().numpy()
  mat = np.vstack((mat,sentence_rep))
 '''

In [0]:
def masked_concat_pool(outputs, mask):
    "Pool MultiBatchEncoder outputs into one vector [last_hidden, max_pool, avg_pool]."
    output = outputs[-1]
    avg_pool = output.masked_fill(mask[:, :, None], 0).mean(dim=1)
    avg_pool *= output.size(1) / (output.size(1)-mask.type(avg_pool.dtype).sum(dim=1))[:,None]
    max_pool = output.masked_fill(mask[:,:,None], -float('inf')).max(dim=1)[0]
    x = torch.cat([output[:,-1], max_pool, avg_pool], 1)
    return x

In [0]:
mbe = MultiBatchEncoder(70,1400,learn_lm.model[0])
xb,yb = learn_lm.data.one_item(resolucoes[0])
sentence = torch.cuda.LongTensor(xb.tolist()[0]).unsqueeze(0)
raw_outputs, outputs, masks = mbe.forward(sentence)
sentence_rep = masked_concat_pool(outputs, masks)
sentence_rep.shape

torch.Size([1, 1200])

In [0]:
mbe = MultiBatchEncoder(70,1400,learn_lm.model[0])
mat = np.zeros(1200)
for resolucao in resolucoes:
  xb,yb = learn_lm.data.one_item(resolucao)
  sentence = torch.cuda.LongTensor(xb.tolist()[0]).unsqueeze(0)
  raw_outputs, outputs, masks = mbe.forward(sentence)
  sentence_rep = masked_concat_pool(outputs, masks)
  mat = np.vstack((mat,sentence_rep))

In [0]:
mat.shape

(866, 1200)

In [0]:
np.save('mat', mat)